# List of Exposures with Hologram at USDF


- work with Weakly_2023_01
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/09/19
- update : 2023/01/20



In [1]:
import sys
print(sys.executable)

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-5.1.0/bin/python3


In [2]:
print(sys.path)

['/home/d/dagoret/notebooks/AuxTelComm/notebooks_usdf/butlertools', '/home/d/dagoret/repos/repos_w_2023_01/Spectractor/lib/python', '/home/d/dagoret/repos/repos_w_2023_01/atmospec/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-5.1.0/Linux64/obs_lsst/g81f1fedfbb+e5435568f8/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-5.1.0/Linux64/pipe_tasks/g67fd32f368+06ff137360/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-5.1.0/Linux64/shapelet/g082f9a06f4+23c005d4a7/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-5.1.0/Linux64/meas_modelfit/ge3ed7fa540+fbc60da186/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-5.1.0/Linux64/meas_extensions_shapeHSM/g9e404089df+ce4c866f0f/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-5.1.0/Linux64/scarlet_extensions/g9d18589735+87f79c168d/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-5.1.0/Linux64/proxmin/g33b4157f25/lib/python', '/opt/lsst/software

In [3]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2023_01/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-5.1.0/eups 	setup
spectractor           LOCAL:/home/d/dagoret/repos/repos_w_2023_01/Spectractor 	setup


In [4]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

w_2023_01
   g754a7f0350+935911ccad 	current w_2023_01 setup


In [5]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits
from astropy.time import Time

In [6]:
from IPython.display import display, HTML

In [7]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [8]:
import lsst.daf.butler as dafButler

In [9]:
#repo = "/sdf/group/rubin/repo/main"
repo="/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [10]:
collection='LATISS/raw/all'

## List of Exposures

In [11]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num','time_start','time_end' ,'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle','science_program','jd','mjd'])

In [12]:
for count, info in enumerate(registry.queryDimensionRecords('exposure',where= "instrument='LATISS'")):
    
    
    #df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,info.timespan.begin.to_string(),info.timespan.end.to_string() ,info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
    #                         info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle, info.science_program]
    df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,pd.to_datetime(info.timespan.begin.to_string()),pd.to_datetime(info.timespan.end.to_string()) ,info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle, info.science_program,
                             info.timespan.begin.jd,info.timespan.begin.mjd ]
    
    if count < 5:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
        
        mjd = Time(info.timespan.begin.to_string()).mjd
        jd = Time(info.timespan.begin.to_string()).jd
        print(mjd,jd)
    

----------------------------------------------------- 0 ---------------------------------------------------------
exposure:
  instrument: 'LATISS'
  id: 2022083100004
  physical_filter: 'unknown~unknown'
  obs_id: 'AT_O_20220831_000004'
  exposure_time: 0.0
  dark_time: 0.0160482
  observation_type: 'bias'
  observation_reason: 'bias'
  day_obs: 20220831
  seq_num: 4
  seq_start: 4
  seq_end: 4
  group_name: '2022083100004'
  group_id: 2022083100004
  target_name: 'UNKNOWN'
  science_program: 'unknown'
  tracking_ra: None
  tracking_dec: None
  sky_angle: None
  azimuth: None
  zenith_angle: None
  has_simulated: False
  timespan: Timespan(begin=astropy.time.Time('2022-08-31 22:58:13.633984', scale='tai', format='iso'), end=astropy.time.Time('2022-08-31 22:58:13.650000', scale='tai', format='iso'))
	 id:                   2022083100004
	 day_obs:              20220831
	 seq_num:              4
	 type-of-observation:  bias
	 target:               UNKNOWN
59822.957102245186 2459823.45710

In [13]:
info.toDict()

{'instrument': 'LATISS',
 'id': 2023011900405,
 'physical_filter': 'OG550_65mm_1~holo4_003',
 'obs_id': 'AT_O_20230119_000405',
 'exposure_time': 30.0,
 'dark_time': 30.2268,
 'observation_type': 'science',
 'observation_reason': 'object',
 'day_obs': 20230119,
 'seq_num': 405,
 'seq_start': 405,
 'seq_end': 405,
 'group_name': '2023-01-20T04:59:16.243',
 'group_id': 2541203562430000,
 'target_name': 'HD111980',
 'science_program': 'spec',
 'tracking_ra': 193.301508006012,
 'tracking_dec': -18.490055907875227,
 'sky_angle': 334.6912307930275,
 'azimuth': 96.75340302707627,
 'zenith_angle': 63.02832582750125,
 'has_simulated': False,
 'timespan': Timespan(begin=astropy.time.Time('2023-01-20 05:04:19.522001', scale='tai', format='iso'), end=astropy.time.Time('2023-01-20 05:04:49.749000', scale='tai', format='iso'))}

In [14]:
df_exposure

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,science_program,jd,mjd
0,2022083100004,AT_O_20220831_000004,20220831,4,2022-08-31 22:58:13.633984,2022-08-31 22:58:13.650,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957102
1,2022083100008,AT_O_20220831_000008,20220831,8,2022-08-31 22:59:16.387013,2022-08-31 22:59:16.398,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957829
2,2022083100007,AT_O_20220831_000007,20220831,7,2022-08-31 22:59:11.978990,2022-08-31 22:59:11.991,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957778
3,2022083100009,AT_O_20220831_000009,20220831,9,2022-08-31 22:59:20.801996,2022-08-31 22:59:20.813,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957880
4,2022083100012,AT_O_20220831_000012,20220831,12,2022-08-31 23:00:00.113994,2022-08-31 23:00:03.124,dark,UNKNOWN,unknown~unknown,NaN,3.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.958335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32692,2023011900404,AT_O_20230119_000404,20230119,404,2023-01-20 05:03:33.939992,2023-01-20 05:04:04.173,science,HD111980,BG40_65mm_1~holo4_003,63.193068,30.0,193.301522,-18.489999,334.691263,spec,2.459965e+06,59964.210809
32693,2023011900406,AT_O_20230119_000406,20230119,406,2023-01-20 05:05:02.351984,2023-01-20 05:05:32.591,science,HD111980,OG550_65mm_1~holo4_003,62.874319,30.0,193.301477,-18.490025,334.691194,spec,2.459965e+06,59964.211833
32694,2023011900401,AT_O_20230119_000401,20230119,401,2023-01-20 05:01:23.856006,2023-01-20 05:01:54.083,science,HD111980,empty~holo4_003,63.65836,30.0,193.301525,-18.490037,334.691242,spec,2.459965e+06,59964.209304
32695,2023011900403,AT_O_20230119_000403,20230119,403,2023-01-20 05:02:51.134993,2023-01-20 05:03:21.364,science,HD111980,BG40_65mm_1~holo4_003,63.347052,30.0,193.301575,-18.489994,334.691289,spec,2.459965e+06,59964.210314


In [15]:
df_exposure = df_exposure.astype({"id": int,'day_obs': int,'seq_num':int})

In [16]:
df_exposure

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,science_program,jd,mjd
0,2022083100004,AT_O_20220831_000004,20220831,4,2022-08-31 22:58:13.633984,2022-08-31 22:58:13.650,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957102
1,2022083100008,AT_O_20220831_000008,20220831,8,2022-08-31 22:59:16.387013,2022-08-31 22:59:16.398,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957829
2,2022083100007,AT_O_20220831_000007,20220831,7,2022-08-31 22:59:11.978990,2022-08-31 22:59:11.991,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957778
3,2022083100009,AT_O_20220831_000009,20220831,9,2022-08-31 22:59:20.801996,2022-08-31 22:59:20.813,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957880
4,2022083100012,AT_O_20220831_000012,20220831,12,2022-08-31 23:00:00.113994,2022-08-31 23:00:03.124,dark,UNKNOWN,unknown~unknown,NaN,3.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.958335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32692,2023011900404,AT_O_20230119_000404,20230119,404,2023-01-20 05:03:33.939992,2023-01-20 05:04:04.173,science,HD111980,BG40_65mm_1~holo4_003,63.193068,30.0,193.301522,-18.489999,334.691263,spec,2.459965e+06,59964.210809
32693,2023011900406,AT_O_20230119_000406,20230119,406,2023-01-20 05:05:02.351984,2023-01-20 05:05:32.591,science,HD111980,OG550_65mm_1~holo4_003,62.874319,30.0,193.301477,-18.490025,334.691194,spec,2.459965e+06,59964.211833
32694,2023011900401,AT_O_20230119_000401,20230119,401,2023-01-20 05:01:23.856006,2023-01-20 05:01:54.083,science,HD111980,empty~holo4_003,63.65836,30.0,193.301525,-18.490037,334.691242,spec,2.459965e+06,59964.209304
32695,2023011900403,AT_O_20230119_000403,20230119,403,2023-01-20 05:02:51.134993,2023-01-20 05:03:21.364,science,HD111980,BG40_65mm_1~holo4_003,63.347052,30.0,193.301575,-18.489994,334.691289,spec,2.459965e+06,59964.210314


## Selection of science

In [17]:
df_science = df_exposure[df_exposure.type == 'science']

In [18]:
df_science.sort_values(by="id",ascending=True,inplace=True)

/tmp/ipykernel_36238/3986782663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science.sort_values(by="id",ascending=True,inplace=True)


In [19]:
df_science.tail(60)

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,science_program,jd,mjd
32585,2023011900314,AT_O_20230119_000314,20230119,314,2023-01-20 03:58:58.903999,2023-01-20 03:59:29.133,science,HD37962,empty~holo4_003,19.407923,30.0,85.205404,-31.398154,182.626417,spec,2.459965e+06,59964.165960
32616,2023011900315,AT_O_20230119_000315,20230119,315,2023-01-20 03:59:41.898013,2023-01-20 04:00:12.131,science,HD37962,empty~holo4_003,19.561108,30.0,85.205405,-31.398032,182.626837,spec,2.459965e+06,59964.166457
32586,2023011900316,AT_O_20230119_000316,20230119,316,2023-01-20 04:00:26.431990,2023-01-20 04:00:56.661,science,HD37962,BG40_65mm_1~holo4_003,19.721365,30.0,85.205444,-31.398188,182.626323,spec,2.459965e+06,59964.166973
32617,2023011900317,AT_O_20230119_000317,20230119,317,2023-01-20 04:01:09.414014,2023-01-20 04:01:39.658,science,HD37962,BG40_65mm_1~holo4_003,19.874524,30.0,85.205444,-31.398272,182.626126,spec,2.459965e+06,59964.167470
32587,2023011900318,AT_O_20230119_000318,20230119,318,2023-01-20 04:01:55.002018,2023-01-20 04:02:25.238,science,HD37962,OG550_65mm_1~holo4_003,20.034831,30.0,85.205421,-31.398036,182.626807,spec,2.459965e+06,59964.167998
32618,2023011900319,AT_O_20230119_000319,20230119,319,2023-01-20 04:02:37.871993,2023-01-20 04:03:08.098,science,HD37962,OG550_65mm_1~holo4_003,20.187968,30.0,85.205428,-31.398038,182.626664,spec,2.459965e+06,59964.168494
32591,2023011900326,AT_O_20230119_000326,20230119,326,2023-01-20 04:09:09.539995,2023-01-20 04:09:39.776,science,HD185975,empty~holo4_003,62.163814,30.0,306.148905,-87.479588,253.460331,spec,2.459965e+06,59964.173027
32622,2023011900327,AT_O_20230119_000327,20230119,327,2023-01-20 04:09:52.460020,2023-01-20 04:10:22.702,science,HD185975,empty~holo4_003,62.166262,30.0,306.148241,-87.479593,253.460927,spec,2.459965e+06,59964.173524
32592,2023011900328,AT_O_20230119_000328,20230119,328,2023-01-20 04:10:41.802017,2023-01-20 04:11:12.042,science,HD185975,BG40_65mm_1~holo4_003,62.169024,30.0,306.152464,-87.479536,253.456646,spec,2.459965e+06,59964.174095
32623,2023011900329,AT_O_20230119_000329,20230119,329,2023-01-20 04:11:24.662980,2023-01-20 04:11:54.896,science,HD185975,BG40_65mm_1~holo4_003,62.171414,30.0,306.142905,-87.479652,253.465892,spec,2.459965e+06,59964.174591


In [20]:
df_science.day_obs.unique()

array([20220912, 20220913, 20220914, 20220927, 20220928, 20220929,
       20221011, 20221012, 20221013, 20221025, 20221026, 20221027,
       20221108, 20221109, 20221110, 20221122, 20221123, 20221124,
       20221207, 20221208, 20221209, 20221210, 20221212, 20230117,
       20230118, 20230119])

## Selection of date

2021-02-16  2021-02-18  2021-06-08  2021-06-10  2021-07-07  2021-09-07  2021-09-09  2021-10-06  2021-11-02  2021-11-04  2022-02-16  2022-03-15  2022-03-17  20220502  
2021-02-17  2021-03-11  2021-06-09  2021-07-06  2021-07-08  2021-09-08  2021-10-05  2021-10-07  2021-11-03  2022-02-15  2022-02-17  2022-03-16  2022-03-18  20220503 20220524 20220607 20220608 20220609

### Add those

20220504 - Not existing

In [59]:
#date_sel = 20220630
#date_sel = 20220630
#date_sel = 20221208
#date_sel = 20221209

# data in january
date_sel= 20230119

In [60]:
df_science_selected = df_science[df_science.day_obs == date_sel ]

In [61]:
len(df_science_selected)

132

# List of filters

In [62]:
list_of_filters = df_science_selected['filter'].unique()
list_of_filters

array(['empty~holo4_003', 'BG40_65mm_1~holo4_003',
       'OG550_65mm_1~holo4_003'], dtype=object)

In [63]:
selected_filters = []
for filt in list_of_filters:
    #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
    flag_sel = (filt.find('holo4') != -1) 
    #flag_sel = (filt.find('ronchi170') != -1) 
    if flag_sel:
        selected_filters.append(filt) 
selected_filters = np.array(selected_filters)
selected_filters       

array(['empty~holo4_003', 'BG40_65mm_1~holo4_003',
       'OG550_65mm_1~holo4_003'], dtype='<U22')

## Selection of filter

- But now we have a loop on filters

In [64]:
#idx_sel_filt = 0
#selected_filter = selected_filters[0]
#selected_filter

## Final selection of exposure

- loop on selected filters

In [65]:
pd.set_option('display.max_rows', None)

In [66]:
all_def_science_selected_final = []

for selected_filter in selected_filters:

    print(selected_filter)
    df_science_selected_final = df_science_selected[df_science_selected["filter"] ==  selected_filter]
    
    #print(df_science_selected_final)
    all_def_science_selected_final.append(df_science_selected_final.copy())
    
    df = df_science_selected_final
    df_sel = df[['day_obs', 'seq_num']]
    
    
    NN = len(df)
    
    if NN>0:
        # build the line
        
        df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
    
    
         # prepare output
        filename_out = "visitdispersers_"+ str(date_sel) + "_filt_" + selected_filter.replace('~','-') + ".list"
        print(filename_out)
        
        df_out = df_sel[["line"]]
        df_out.to_csv(filename_out, index=False, header=None)

empty~holo4_003
visitdispersers_20230119_filt_empty-holo4_003.list
BG40_65mm_1~holo4_003
visitdispersers_20230119_filt_BG40_65mm_1-holo4_003.list
OG550_65mm_1~holo4_003
visitdispersers_20230119_filt_OG550_65mm_1-holo4_003.list


/tmp/ipykernel_36238/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
/tmp/ipykernel_36238/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
/tmp/ipykernel_36238/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [67]:
N = len(all_def_science_selected_final)

In [68]:
if N>=1:
    n0=len(all_def_science_selected_final[0])
    display(all_def_science_selected_final[0])

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,science_program,jd,mjd
32447,2023011900162,AT_O_20230119_000162,20230119,162,2023-01-20 01:00:57.548015,2023-01-20 01:01:27.783,science,HD185975,empty~holo4_003,60.782103,30.0,306.587964,-87.514407,207.154859,spec,2.459965e+06,59964.042333
32478,2023011900163,AT_O_20230119_000163,20230119,163,2023-01-20 01:01:40.536999,2023-01-20 01:02:10.777,science,HD185975,empty~holo4_003,60.789304,30.0,306.588403,-87.514434,207.154182,spec,2.459965e+06,59964.042830
32456,2023011900180,AT_O_20230119_000180,20230119,180,2023-01-20 01:16:29.940986,2023-01-20 01:17:00.175,science,HD37962,empty~holo4_003,15.508433,30.0,85.222929,-31.313015,359.198323,spec,2.459965e+06,59964.053124
32487,2023011900181,AT_O_20230119_000181,20230119,181,2023-01-20 01:17:12.778010,2023-01-20 01:17:43.015,science,HD37962,empty~holo4_003,15.35501,30.0,85.222964,-31.313009,359.198329,spec,2.459965e+06,59964.053620
32491,2023011900189,AT_O_20230119_000189,20230119,189,2023-01-20 01:21:52.976007,2023-01-20 01:22:23.211,science,HD37962,empty~holo4_003,14.356285,30.0,85.223625,-31.313105,0.051256,spec,2.459965e+06,59964.056863
32461,2023011900190,AT_O_20230119_000190,20230119,190,2023-01-20 01:22:35.896997,2023-01-20 01:23:06.127,science,HD37962,empty~holo4_003,14.202788,30.0,85.223586,-31.313099,0.051084,spec,2.459965e+06,59964.057360
32465,2023011900198,AT_O_20230119_000198,20230119,198,2023-01-20 01:28:47.949013,2023-01-20 01:29:18.183,science,HD031128,empty~holo4_003,3.992191,30.0,73.075039,-27.095449,130.960064,spec,2.459965e+06,59964.061666
32496,2023011900199,AT_O_20230119_000199,20230119,199,2023-01-20 01:29:30.756988,2023-01-20 01:30:00.989,science,HD031128,empty~holo4_003,3.89608,30.0,73.074976,-27.095502,130.960432,spec,2.459965e+06,59964.062162
32536,2023011900217,AT_O_20230119_000217,20230119,217,2023-01-20 01:45:34.527017,2023-01-20 01:46:04.754,science,HD185975,empty~holo4_003,61.221301,30.0,306.367311,-87.493637,218.185978,spec,2.459965e+06,59964.073316
32506,2023011900218,AT_O_20230119_000218,20230119,218,2023-01-20 01:46:17.333987,2023-01-20 01:46:47.575,science,HD185975,empty~holo4_003,61.22762,30.0,306.366908,-87.493629,218.18635,spec,2.459965e+06,59964.073812


In [69]:
if N>=2:
    n1=len(all_def_science_selected_final[1])
    display(all_def_science_selected_final[1])

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,science_program,jd,mjd
32448,2023011900164,AT_O_20230119_000164,20230119,164,2023-01-20 01:02:24.822013,2023-01-20 01:02:55.065,science,HD185975,BG40_65mm_1~holo4_003,60.796701,30.0,306.588189,-87.514432,207.154316,spec,2.459965e+06,59964.043343
32479,2023011900165,AT_O_20230119_000165,20230119,165,2023-01-20 01:03:07.654008,2023-01-20 01:03:37.882,science,HD185975,BG40_65mm_1~holo4_003,60.803926,30.0,306.587922,-87.514419,207.154446,spec,2.459965e+06,59964.043839
32457,2023011900182,AT_O_20230119_000182,20230119,182,2023-01-20 01:17:57.302009,2023-01-20 01:18:27.541,science,HD37962,BG40_65mm_1~holo4_003,15.198047,30.0,85.223037,-31.313008,359.198482,spec,2.459965e+06,59964.054135
32488,2023011900183,AT_O_20230119_000183,20230119,183,2023-01-20 01:18:40.217004,2023-01-20 01:19:10.453,science,HD37962,BG40_65mm_1~holo4_003,15.044493,30.0,85.222924,-31.313008,359.198325,spec,2.459965e+06,59964.054632
32492,2023011900191,AT_O_20230119_000191,20230119,191,2023-01-20 01:23:20.514016,2023-01-20 01:23:50.750,science,HD37962,BG40_65mm_1~holo4_003,14.042277,30.0,85.223693,-31.313097,0.051104,spec,2.459965e+06,59964.057876
32462,2023011900192,AT_O_20230119_000192,20230119,192,2023-01-20 01:24:03.424988,2023-01-20 01:24:33.666,science,HD37962,BG40_65mm_1~holo4_003,13.888781,30.0,85.223654,-31.313103,0.05111,spec,2.459965e+06,59964.058373
32466,2023011900200,AT_O_20230119_000200,20230119,200,2023-01-20 01:30:15.217017,2023-01-20 01:30:45.456,science,HD031128,BG40_65mm_1~holo4_003,3.802079,30.0,73.075007,-27.095437,130.960993,spec,2.459965e+06,59964.062676
32497,2023011900201,AT_O_20230119_000201,20230119,201,2023-01-20 01:30:58.140984,2023-01-20 01:31:28.367,science,HD031128,BG40_65mm_1~holo4_003,3.714488,30.0,73.074953,-27.095485,130.961221,spec,2.459965e+06,59964.063173
32537,2023011900219,AT_O_20230119_000219,20230119,219,2023-01-20 01:47:01.814011,2023-01-20 01:47:32.053,science,HD185975,BG40_65mm_1~holo4_003,61.234351,30.0,306.366897,-87.493635,218.18619,spec,2.459965e+06,59964.074327
32507,2023011900220,AT_O_20230119_000220,20230119,220,2023-01-20 01:47:44.735001,2023-01-20 01:48:14.975,science,HD185975,BG40_65mm_1~holo4_003,61.240761,30.0,306.367111,-87.49364,218.185787,spec,2.459965e+06,59964.074823


In [70]:
if N>=3:
    n2=len(all_def_science_selected_final[2])
    display(all_def_science_selected_final[2])

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,science_program,jd,mjd
32449,2023011900166,AT_O_20230119_000166,20230119,166,2023-01-20 01:03:53.399002,2023-01-20 01:04:23.637,science,HD185975,OG550_65mm_1~holo4_003,60.811615,30.0,306.58803,-87.514434,207.154234,spec,2.459965e+06,59964.044368
32480,2023011900167,AT_O_20230119_000167,20230119,167,2023-01-20 01:04:36.335000,2023-01-20 01:05:06.568,science,HD185975,OG550_65mm_1~holo4_003,60.818624,30.0,306.588304,-87.514432,207.153826,spec,2.459965e+06,59964.044865
32458,2023011900184,AT_O_20230119_000184,20230119,184,2023-01-20 01:19:25.838000,2023-01-20 01:19:56.077,science,HD37962,OG550_65mm_1~holo4_003,14.880287,30.0,85.222879,-31.31301,359.198414,spec,2.459965e+06,59964.055160
32489,2023011900185,AT_O_20230119_000185,20230119,185,2023-01-20 01:20:08.757985,2023-01-20 01:20:38.990,science,HD37962,OG550_65mm_1~holo4_003,14.726889,30.0,85.222953,-31.313009,359.198284,spec,2.459965e+06,59964.055657
32493,2023011900193,AT_O_20230119_000193,20230119,193,2023-01-20 01:24:49.437009,2023-01-20 01:25:19.668,science,HD37962,OG550_65mm_1~holo4_003,13.724597,30.0,85.223637,-31.313109,0.051225,spec,2.459965e+06,59964.058906
32463,2023011900194,AT_O_20230119_000194,20230119,194,2023-01-20 01:25:32.251020,2023-01-20 01:26:02.483,science,HD37962,OG550_65mm_1~holo4_003,13.574646,30.0,85.223572,-31.313111,0.051091,spec,2.459965e+06,59964.059401
32467,2023011900202,AT_O_20230119_000202,20230119,202,2023-01-20 01:31:44.457007,2023-01-20 01:32:14.702,science,HD031128,OG550_65mm_1~holo4_003,3.624217,30.0,73.07503,-27.095473,130.962939,spec,2.459965e+06,59964.063709
32498,2023011900203,AT_O_20230119_000203,20230119,203,2023-01-20 01:32:27.381981,2023-01-20 01:32:57.607,science,HD031128,OG550_65mm_1~holo4_003,3.548572,30.0,73.075014,-27.095463,130.962992,spec,2.459965e+06,59964.064206
32538,2023011900221,AT_O_20230119_000221,20230119,221,2023-01-20 01:48:30.360986,2023-01-20 01:49:00.587,science,HD185975,OG550_65mm_1~holo4_003,61.247476,30.0,306.366924,-87.49363,218.18588,spec,2.459965e+06,59964.075351
32508,2023011900222,AT_O_20230119_000222,20230119,222,2023-01-20 01:49:13.190003,2023-01-20 01:49:43.426,science,HD185975,OG550_65mm_1~holo4_003,61.253865,30.0,306.366875,-87.493632,218.185855,spec,2.459965e+06,59964.075847


In [71]:
if N>=4:
    n3=len(all_def_science_selected_final[3])
    display(all_def_science_selected_final[3])

In [72]:
if N>=5:
    n4=len(all_def_science_selected_final[4])
    display(all_def_science_selected_final[4])

In [73]:
df_forsave = pd.concat(all_def_science_selected_final)

In [74]:
filename_out_csv = "holosummary_"+ str(date_sel) + ".csv"
filename_out_excel = "holosummary_"+ str(date_sel) + ".xlsx"

In [75]:
filename_out_csv

'holosummary_20230119.csv'

In [76]:
df_forsave.to_csv(filename_out_csv) 

In [77]:
df_forsave.to_excel(filename_out_excel) 